In [1]:
# initial run(0) or update run (1)
update = 0
# https://www.rotowire.com/betting/nba/player-props.php

import requests, time, re, os, json, mysql.connector
#import mysql, pymysql, pyodbc
#import sqlalchemy as sal
#from sqlalchemy import create_engine
import pandas as pd
pd.set_option('display.max_columns', 500)
import numpy as np
#from bs4 import BeautifulSoup as bs
from datetime import date

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

#importing credentials from txt file
with open('../../../Notes-General/config.txt', 'r') as f:
    creds = f.read()
creds = json.loads(creds)

# path to downloads folder
download_path = creds['downloads']

#### using the save as button on the page to save the files individually.
#### the rows of the tables that are not visible are hidden from HTML
#### saving is the only way to get all the rows that I know at this time

# path to webdriver for selenium
try:
    service = Service(r"..\browser\geckodriver.exe")
    driver = webdriver.Firefox(service=service)
    waitTime = 10

    site = "https://www.rotowire.com/betting/nba/player-props.php"
    driver.get(site)
except:
    driver.close()

# will be used to loop through the webpage
data_prop_names = ["PTS","REB","AST",
                   "THREES","BLK","STL",
                   "TURNOVERS","PTSREBAST","PTSREB",
                   "PTSAST","REBAST","STLBLK"]
missing = []
for i in range(len(data_prop_names)):
# find the element in the source
    try:
        ele = driver.find_element(By.CSS_SELECTOR, "div[data-prop={}]".format(data_prop_names[i]))
        # retrieve its page coords
        loc = ele.location
        #scroll to the location to activate the table creation
        driver.execute_script("window.scrollTo(0, {});".format(str(loc["y"])))
        # pause for data loading
        time.sleep(3)
        ###### save to csv 
        export_button = ele.find_element(By.CLASS_NAME, "is-csv")
        export_button.click()
    except Exception as e:
        print(data_prop_names[i], e)
        missing.append(i)
        continue
    
driver.close()

# columns to use for renaming csv's
columns = ['PLAYER', 'team', 'opp', 'dk_line', 'dk_oOdds', 'dk_uOdds',
          'fd_line', 'fd_oOdds', 'fd_uOdds','m_line', 'm_oOdds', 'm_uOdds',
          'pb_line', 'pb_oOdds', 'pb_uOdds']
# will be used to get the correct file number when there are missing files
count = 0
# loop through each file to agg into a single data frame
for i in range(0, len(data_prop_names)):
    #uses the indices for the missing odds files
    if i in missing:
        continue
    else:
        stat = data_prop_names[i]
        if i == 0:
            path = download_path + "nba-player-props-rotowire.csv"
            data = pd.read_csv(path, skiprows=1)
            data.columns=columns
            data['stat'] = stat
        else:
            count += 1
            path = download_path + "nba-player-props-rotowire({}).csv".format(count)
            temp = pd.read_csv(path, skiprows=1)
            temp.columns=columns
            temp['stat'] = stat
            # concat data into main df    
            data = pd.concat([data, temp])
        #delete file after use
        os.remove(path)
# using dk line as the default and averaging the other options when DK is not available
data["line"] = np.where(data["dk_line"].isna(), 
                       data[["fd_line","m_line","pb_line"]].mean(axis=1, skipna=True),
                       data["dk_line"])
data["oOdds"] = np.where(data["dk_oOdds"].isna(), 
                       data[["fd_oOdds","m_oOdds","pb_oOdds"]].mean(axis=1, skipna=True),
                       data["dk_oOdds"])
data["uOdds"] = np.where(data["dk_uOdds"].isna(), 
                       data[["fd_uOdds","m_uOdds","pb_uOdds"]].mean(axis=1, skipna=True),
                       data["dk_uOdds"])

data = data[["PLAYER", "team", "stat", "line", "oOdds", "uOdds"]]
# adding the date of the odds
data.loc[:,'date'] = date.today()  

date_string = str(date.today())
# if this is not an update run then the data will be saved. If it is an updated (1) then
# the next cell will need to be executed to filter
if update == 0:
    data.to_csv("../data/{}_odds.csv".format(date_string), index=False)
else:
    # updating previously pulled odds
    # if teams have completed or started their games they need to be filtered 
    # from the newest pull bec the roto site does not accurately save lines once the game start

    # user input to 
    filter_teams = input("Enter Teams Abbrev to filter separated by space >>").split()

    #filtering the input teams from the updated odds data
    data = data[~data['team'].isin(filter_teams)]
    # setting player as index temporarily 
    data.set_index("PLAYER", inplace=True)

    # import the prior odds data saved
    prevOddsData = pd.read_csv("../data/{}_odds.csv".format(date_string), index_col="PLAYER")

    # keep players data in the old data when they aren't present in the new data and then add
    # all of the new data (this achieves the update portion
    data = pd.concat([data[~data.index.isin(prevOddsData.index)], prevOddsData]).reset_index()

    data.to_csv("../data/{}_odds.csv".format(date_string), index=False)

In [ ]:
CHA TOR

In [ ]:
DET NOP DEN BKN CLE TOR ORL OKC LAL WAS CHA SAS
DAL BOS CHI MIN PHI MEM LAL GSW SAC LAC CLE MIA HOU